# Helper Functionality

In [1]:
# Following code is needed to preconfigure this notebook
import datetime
import sys
import os
sys.path.insert(0, os.path.abspath('../../..'))

import pyflow as pf

scratchdir = os.path.join('/', 'path', 'to', 'scratch')
filesdir = os.path.join(scratchdir, 'files')
outdir = os.path.join(scratchdir, 'out')


class CourseSuite(pf.Suite):
    """
    This CourseSuite object will be used throughout the course to provide sensible
    defaults without verbosity
    """
    def __init__(self, name, **kwargs):
        
        config = {
            'host': pf.LocalHost('localhost'),
            'files': os.path.join(filesdir, name),
            'home': outdir,
            'defstatus': pf.state.suspended
        }
        config.update(kwargs)
        
        super().__init__(name, **config)

**pyflow** aims to contain not just a collection of ecflow functionality, but also helper functionality to assist in building suites. Where idiomatic uses of **ecFlow** result in the same mechanisms being built repeatedly, **pyflow** can incorporate these to help generate clearer suites.

The `ecflow_name()` functionality converts an arbitrary string into a name which meets the character restrictions for ecflow nodes. This is very useful for converting strings such as hostnames or the names of various data sets into a form that can be used as the name of a Family or Task.

In [2]:
print(pf.ecflow_name('hyphenated-name'))

hyphenated_name


The `all_complete()` and `sequence()` functions facilitate working with generated sequences of python tasks. `all_complete()` generates an expression suitable for use in triggers (or completes). `sequence()` generates triggers such that all of the tasks will run sequentially.

In [3]:
with CourseSuite('sequences') as s:
    tasks = [pf.Task('t_{}'.format(i)) for i in range(10)]
    pf.Task('done', triggers=pf.all_complete(tasks))
    pf.sequence(tasks)

s

CourseSuite(/sequences)

A common idiom in looping suites is to have two suites that both loop on dates/times, one which runs behind the other. For example the `lag` family running after the forecast has completed. This idiom can be expressed more clearly by encapsulating its functionality.

In [5]:
with pf.Suite('follow') as s:
    with pf.Family('leader') as leader:
        pf.RepeatDate("YMD", datetime.date(2019, 1, 1), datetime.date(2019, 12, 31))
    with pf.Family('follower') as follower:
        pf.RepeatDate("YMD", datetime.date(2019, 1, 1), datetime.date(2019, 12, 31))
    follower.follow = leader.YMD

s

Suite(/follow)

This collection of utility functionality is (perpetually) in progress, and will be updated to account for useful idioms as they emerge.